<a href="https://colab.research.google.com/github/dmlcks123/3_3_Other_Considerations_in_the_Regression_Model/blob/main/Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lasso

0. OLS 추정 중 최적의 모델을 찾아가는 과정 --> 불편성과 분산의 절충 --> 검정오차을 가장 낮추는 모델 --> 모델 예측도 향상

1. 설명변수가 추가되면 R_square가 올라감(RSS 증가), 설명변수가 늘어나는 것에 규제 필요 
* 이전 방식 : adj_R_square --> 모델정확도 판단의 근거는 되지만 최적모델을 제시하지 못함

2. 모델 적합시 설명변수가 늘어나는 것의 문제: 데이터 과적합(overfitting) 발생
* 실제모델이 변수가 적은 모델이라면 더 큰 문제

3. 데이터 과적합 : 훈련 MSE < 검정 MSE --> 훈련 MSE는 작지만 검정 MSE는 커짐 
* 편향은 줄지만 분산이 커지는 문제
* MSE(mean square error) = $\frac{1}{n}\sum_{i=1}^{n} (y_{i}-\hat{f}(x))^2$ --> 예측오차를 줄여나가는 과정

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

In [ ]:
from IPython.display import Image
Image('/gdrive/My Drive/g3.png')

4. 훈련데이터에 대한 모델적합도를 조절하고 검정데이터에 대한 모델적합도를 개선시키는 방안이 필요
* 모든 설명변수를 모델에 적합하는 것을 제한하기 위해 RSS 게산시 패널티 부여
* min $\beta$, $RSS + \lambda \sum_{j=1}^{p}\left | \beta _{j} \right |$
* $\sum_{i=1}^{n}(y_{i} - \beta_0 - \sum_{j=1}^{P}\beta_{j}x_{ij}) + \lambda \sum_{j=1}^{p}\left | \beta _{j} \right |$

5. Ridge와 다른점
* 실제모델이 2변수 모형일 때 검정MSE(Lasso) < 검정MSE(Ridge) --> Lasso 우세
* 실제모델이 45개 변수 모형일 때 분산(Lasso) > 분산(Ridge) --> Ridge 우세
* 람다가 커지면 규제항으로 인해 베타값이 0이 될수 있음

In [ ]:
from IPython.display import Image
Image('/gdrive/My Drive/g4.png')
Image('/gdrive/My Drive/g5.png')

질문. 어떤 방식으로 규제항으로 인해 beta값이 0이 되는지 / 규제항은 어떻게 산출된것인지

#실습

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore') # 경고메시지 숨기기

import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso

In [ ]:
url = 'http://raw.githubusercontent.com/RAhyunki/dataset/main/Default.csv'
Default = pd.read_csv(url)
print(Default.shape)
print(Default.head())

In [ ]:
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
print('Data View')
display(pd.concat([pd.DataFrame(y, columns=['diabetes_value']), 
                   pd.DataFrame(X, columns = diabetes.feature_names)], axis = 1).head())

In [ ]:
alpha = 0.5

#Ridge model
fit = Ridge(alpha = alpha, fit_intercept = True, normalize = True,
            random_state = 123).fit(X, y)
pd.DataFrame(np.hstack([fit.intercept_, fit.coef_]), columns=['alpha = {}'.format(alpha)])


In [ ]:
#Lasso model
fit = Lasso(alpha = alpha, fit_intercept = True, normalize = True,
            random_state = 123).fit(X, y)
pd.DataFrame(np.hstack([fit.intercept_, fit.coef_]), columns=['alpha = {}'.format(alpha)])

# 일부 변수의 계수값 0으로 조절

In [ ]:
result_Ridge = pd.DataFrame()
alpha_candidate = np.hstack([0, np.logspace(-2,1,4)])
for alpha in alpha_candidate:
  fit = Ridge(alpha = alpha, fit_intercept=True, normalize=True, random_state= 123).fit(X,y)
  result_coef = pd.DataFrame(np.hstack([fit.intercept_, fit.coef_]), columns = ['alpha = {}'.format(alpha)])
  result_Ridge = pd.concat([result_Ridge, result_coef], axis = 1)

result_Lasso = pd.DataFrame()
alpha_candidate = np.hstack([0, np.logspace(-2,1,4)])
for alpha in alpha_candidate:
  fit = Lasso(alpha = alpha, fit_intercept=True, normalize=True, random_state= 123).fit(X,y)
  result_coef = pd.DataFrame(np.hstack([fit.intercept_, fit.coef_]), columns = ['alpha = {}'.format(alpha)])
  result_Lasso = pd.concat([result_Lasso, result_coef], axis = 1)

In [ ]:
plt.figure(figsize=(20,20))
result_Ridge.T.plot(legend = False, ax = plt.subplot(211))
plt.title('Ridge')
plt.xticks(np.arange(len(result_Ridge.columns)), [i for i in result_Ridge.columns])
plt.ylabel('coefficients')
plt.grid()

plt.figure(figsize=(20,20))
result_Lasso.T.plot(legend = False, ax = plt.subplot(211))
plt.title('Lasso')
plt.xticks(np.arange(len(result_Lasso.columns)), [i for i in result_Lasso.columns])
plt.ylabel('coefficients')
plt.grid()